In [ ]:
!pip install torch transformers datasets torchvision

In [106]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model ="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model).to("cuda" if torch.cuda.is_available() else "cpu")

In [108]:
text= "I feel not good"
inputs = tokenizer(text,return_tensors="pt").to(model.device) #text to numeric value

In [110]:
with torch.no_grad(): #The model operates in inference mode and does not perform training or gradient computation
    outputs = model(**inputs) 
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()



In [112]:
etiketler = ["Negative", "Positive"]
print("Tahmin:", etiketler[prediction])

Tahmin: Negative


In [126]:
import time 
start = time.time()
with torch.no_grad():
    output = model(**inputs)
end = time.time()

print(end-start)

0.016952991485595703


In [128]:
#original model size
import os
torch.save(model.state_dict(),"origin_model.pt")
print(os.path.getsize("origin_model.pt")/1e6,"MB")

267.860606 MB


In [118]:
from torch.quantization import quantize_dynamic

quantized_model = quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype = torch.qint8
)

In [120]:
import time 
start = time.time()
with torch.no_grad():
    output = quantized_model(**inputs)
end = time.time()

print(end-start)

0.010969400405883789


In [122]:
#quantized model size
import os
torch.save(quantized_model.state_dict(),"quantized_model.pt")
print(os.path.getsize("quantized_model.pt")/1e6,"MB")

138.715658 MB
